<a href="https://colab.research.google.com/github/RuiminYan/CubeRoot/blob/master/RubiksSolverApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RubiksSolver web app
###For the detail, please see https://github.com/or18/RubiksSolver

In [1]:
#@title **More practical app with realtime output, stop button and timeout (For Google Drive users)**
#@markdown Run this cell to launch the gradio GUI.<br>Note: if the tables have not been created yet, it takes about 3 to 4 minutes to launch the GUI.

import os
os.chdir('/content')

!wget https://downloads.python.org/pypy/pypy3.10-v7.3.16-linux64.tar.bz2
!tar -xf pypy3.10-v7.3.16-linux64.tar.bz2
!rm pypy3.10-v7.3.16-linux64.tar.bz2
os.environ['PATH']+=':/content/pypy3.10-v7.3.16-linux64/bin/'
!pip install gradio

from google.colab import drive
import subprocess
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')
if not os.path.exists('/content/drive/MyDrive/RubiksSolver'):
    subprocess.run(['git', 'clone', 'https://github.com/or18/RubiksSolver.git'])
%cd /content/drive/MyDrive/RubiksSolver

subprocess.run(['pypy3.10', 'create_all_table.py'])

import RubiksSolver.solver as sv
import gradio as gr
import time
import threading
import math
is_processing=False
lock=threading.Lock()

def run_solver(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, state):
    command = ['pypy3.10', '-u', 'solver_for_app.py', operation, str(scramble), str(rot), str(slot1), str(slot2), str(slot3), str(slot4), str(length), restrict]
    state['process'] = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, universal_newlines=True)
    while not state['stop']:
        output = state['process'].stdout.readline()
        if output== '' and state['process'].poll() is not None:
            state['stop']=True
            break

        if output:
            state['stdout_lines'].append(output.strip())

        if len(state['stdout_lines'])>=m:
            state['process'].terminate()
            state['stop']=True
            break
    state['process'].wait()

def cancel(state):
    if is_processing and state['process'] is None:
        gr.Warning("Other process is running now. Please wait.")
    elif is_processing and state['process'].poll() is None:
        gr.Warning("Process will be terminated soon.")
        state['stop']=True
        state['process'].terminate()
    elif not is_processing:
        gr.Info("No process is running now.")

def timeout(start, t, state):
    while not state['stop']:
        time.sleep(1)
        if time.time()-start>t:
            state['stop']=True
            state['process'].terminate()
            break

def main(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state):
    global is_processing
    with lock:
        if is_processing:
            gr.Warning('Request was rejected')
            return "Other process is running now. Please wait."
    with lock:
        is_processing = True
    if rot is None:
        rot = "N"
    if restrict is None:
        restrict = "UDLRFB"
    start_time = time.time()
    state['stdout_lines']=[]
    state['stop']=False
    timeout_checking=threading.Thread(target=timeout, args=(start_time, t, state), daemon=True)
    solve=threading.Thread(target=run_solver, args=(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, state), daemon=True)
    solve.start()
    timeout_checking.start()
    gr.Info("search start")
    while not state['stop']:
        yield "\n".join(state['stdout_lines']), math.ceil(start_time+t-time.time()), len(state['stdout_lines'])
        time.sleep(s)
    solve.join()
    timeout_checking.join()
    state['process']=None
    state['stop']=None
    yield "\n".join(state['stdout_lines']), math.ceil(start_time+t-time.time()), len(state['stdout_lines'])
    with lock:
        is_processing = False
    gr.Info("Process was finished.")

with gr.Blocks() as interface:
    state = gr.State({
        "process": None,
        "stdout_lines": None,
        "stop": None
    })
    gr.Markdown("# RubiksSolver")
    gr.Markdown("GitHub Repository: [GitHub](https://github.com/or18/RubiksSolver)")
    gr.Markdown("Google colab note book: [Google colab](https://colab.research.google.com/github/or18/RubiksSolver/blob/main/RubiksSolverApp.ipynb)")
    with gr.Tab('Cross & F2L') as tab1:
        gr.Markdown("### Solver for Cross, xcross, xxcross, xxxcross, xxxxcross, F2L, multi slotting, etc.")
        operation = gr.Textbox(value="F2L_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B' D L2 U F2 L2 U' B2 F2 D2 R2 B2 F R2 U2 R U2 B D L2 R'")
        rot = gr.Textbox(label="pre rotation", value="z2")
        with gr.Row():
            slot1 = gr.Checkbox(label="BL", value=True)
            slot2 = gr.Checkbox(label="BR", value=False)
            slot3 = gr.Checkbox(label="FR", value=False)
            slot4 = gr.Checkbox(label="FL", value=False)
        length = gr.Number(label="max length", value=9, minimum=1)
        restrict = gr.Textbox(label="move restriction", value="UDLRFB")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer substeps'):
        gr.Markdown("### Solver for OLL, COLL, ZBLS, VHLS, etc.")
        operation = gr.Textbox(value="LL_substep_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="R2 B2 R2 U2 R' U2 R' B2 R2 U R' F' U F R")
        rot = gr.Textbox(label="pre rotation", value="")
        with gr.Row():
            slot1 = gr.Checkbox(label="CP", value=False)
            slot2 = gr.Checkbox(label="CO", value=True)
            slot3 = gr.Checkbox(label="EP", value=False)
            slot4 = gr.Checkbox(label="EO", value=True)
        length = gr.Number(label="max length", value=12)
        restrict = gr.Textbox(label="move restriction", value="UDRF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="U L2 D' B2 U B2 L2 B2 D F' D' B2 D F' U' F2")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=14)
        restrict = gr.Textbox(label="move restriction", value="URF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer & AUF'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_AUF_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B2 D2 L2 B2 R2 U' B2 D' L2 U L2 U' B F' D2 B F'")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=15)
        restrict = gr.Textbox(label="move restriction", value="URF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

interface.queue(concurrency_count=None)
interface.launch(share=True, show_error=True, show_api=False)

--2025-12-06 16:31:02--  https://downloads.python.org/pypy/pypy3.10-v7.3.16-linux64.tar.bz2
Resolving downloads.python.org (downloads.python.org)... 151.101.0.223, 151.101.64.223, 151.101.128.223, ...
Connecting to downloads.python.org (downloads.python.org)|151.101.0.223|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32651932 (31M) [application/x-tar]
Saving to: ‘pypy3.10-v7.3.16-linux64.tar.bz2’

pypy3.10-v7.3.16-li 100%[===================>]  31.14M  --.-KB/s    in 0.1s    

2025-12-06 16:31:04 (231 MB/s) - ‘pypy3.10-v7.3.16-linux64.tar.bz2’ saved [32651932/32651932]

Mounted at /content/drive
/content/drive/MyDrive/RubiksSolver


TypeError: Blocks.queue() got an unexpected keyword argument 'concurrency_count'

In [ ]:
#@title **More practical app with realtime output, stop button and timeout (For People who don't use Google Drive)**
#@markdown Run this cell to launch the gradio GUI.<br>Note: if the tables have not been created yet, it takes about 3 to 4 minutes to launch the GUI.

import os
os.chdir('/content')

!wget https://downloads.python.org/pypy/pypy3.10-v7.3.16-linux64.tar.bz2
!tar -xf pypy3.10-v7.3.16-linux64.tar.bz2
!rm pypy3.10-v7.3.16-linux64.tar.bz2
os.environ['PATH']+=':/content/pypy3.10-v7.3.16-linux64/bin/'
!pip install gradio

import subprocess
if not os.path.exists('/content/RubiksSolver'):
    subprocess.run(['git', 'clone', 'https://github.com/or18/RubiksSolver.git'])
%cd /content/RubiksSolver

subprocess.run(['pypy3.10', 'create_all_table.py'])

import RubiksSolver.solver as sv
import gradio as gr
import time
import threading
import math
is_processing=False
lock=threading.Lock()

def run_solver(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, state):
    command = ['pypy3.10', '-u', 'solver_for_app.py', operation, str(scramble), str(rot), str(slot1), str(slot2), str(slot3), str(slot4), str(length), restrict]
    state['process'] = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, bufsize=1, universal_newlines=True)
    while not state['stop']:
        output = state['process'].stdout.readline()
        if output== '' and state['process'].poll() is not None:
            state['stop']=True
            break

        if output:
            state['stdout_lines'].append(output.strip())

        if len(state['stdout_lines'])>=m:
            state['process'].terminate()
            state['stop']=True
            break
    state['process'].wait()

def cancel(state):
    if is_processing and state['process'] is None:
        gr.Warning("Other process is running now. Please wait.")
    elif is_processing and state['process'].poll() is None:
        gr.Warning("Process will be terminated soon.")
        state['stop']=True
        state['process'].terminate()
    elif not is_processing:
        gr.Info("No process is running now.")

def timeout(start, t, state):
    while not state['stop']:
        time.sleep(1)
        if time.time()-start>t:
            state['stop']=True
            state['process'].terminate()
            break

def main(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state):
    global is_processing
    with lock:
        if is_processing:
            gr.Warning('Request was rejected')
            return "Other process is running now. Please wait."
    with lock:
        is_processing = True
    if rot is None:
        rot = "N"
    if restrict is None:
        restrict = "UDLRFB"
    start_time = time.time()
    state['stdout_lines']=[]
    state['stop']=False
    timeout_checking=threading.Thread(target=timeout, args=(start_time, t, state), daemon=True)
    solve=threading.Thread(target=run_solver, args=(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, state), daemon=True)
    solve.start()
    timeout_checking.start()
    gr.Info("search start")
    while not state['stop']:
        yield "\n".join(state['stdout_lines']), math.ceil(start_time+t-time.time()), len(state['stdout_lines'])
        time.sleep(s)
    solve.join()
    timeout_checking.join()
    state['process']=None
    state['stop']=None
    yield "\n".join(state['stdout_lines']), math.ceil(start_time+t-time.time()), len(state['stdout_lines'])
    with lock:
        is_processing = False
    gr.Info("Process was finished.")

with gr.Blocks() as interface:
    state = gr.State({
        "process": None,
        "stdout_lines": None,
        "stop": None
    })
    gr.Markdown("# RubiksSolver")
    gr.Markdown("GitHub Repository: [GitHub](https://github.com/or18/RubiksSolver)")
    gr.Markdown("Google colab note book: [Google colab](https://colab.research.google.com/github/or18/RubiksSolver/blob/main/RubiksSolverApp.ipynb)")
    with gr.Tab('Cross & F2L') as tab1:
        gr.Markdown("### Solver for Cross, xcross, xxcross, xxxcross, xxxxcross, F2L, multi slotting, etc.")
        operation = gr.Textbox(value="F2L_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B' D L2 U F2 L2 U' B2 F2 D2 R2 B2 F R2 U2 R U2 B D L2 R'")
        rot = gr.Textbox(label="pre rotation", value="z2")
        with gr.Row():
            slot1 = gr.Checkbox(label="BL", value=True)
            slot2 = gr.Checkbox(label="BR", value=False)
            slot3 = gr.Checkbox(label="FR", value=False)
            slot4 = gr.Checkbox(label="FL", value=False)
        length = gr.Number(label="max length", value=9, minimum=1)
        restrict = gr.Textbox(label="move restriction", value="UDLRFB")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer substeps'):
        gr.Markdown("### Solver for OLL, COLL, ZBLS, VHLS, etc.")
        operation = gr.Textbox(value="LL_substep_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="R2 B2 R2 U2 R' U2 R' B2 R2 U R' F' U F R")
        rot = gr.Textbox(label="pre rotation", value="")
        with gr.Row():
            slot1 = gr.Checkbox(label="CP", value=False)
            slot2 = gr.Checkbox(label="CO", value=True)
            slot3 = gr.Checkbox(label="EP", value=False)
            slot4 = gr.Checkbox(label="EO", value=True)
        length = gr.Number(label="max length", value=12)
        restrict = gr.Textbox(label="move restriction", value="UDRF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="U L2 D' B2 U B2 L2 B2 D F' D' B2 D F' U' F2")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=14)
        restrict = gr.Textbox(label="move restriction", value="URF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

    with gr.Tab('Last Layer & AUF'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_AUF_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B2 D2 L2 B2 R2 U' B2 D' L2 U L2 U' B F' D2 B F'")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=15)
        restrict = gr.Textbox(label="move restriction", value="URF")
        with gr.Row():
            t = gr.Number(label="timeout", value=10, minimum=0)
            s = gr.Number(label="output refresh rate", value=1, minimum=0.1)
            m = gr.Number(label="maximum number of output", value=20, minimum=0, maximum=1000000)
        with gr.Row():
            button = gr.Button("start")
            stop_btn = gr.Button("stop")
        with gr.Row():
            limit=gr.Number(label="time limit", interactive=False)
            n = gr.Number(label="output lines", interactive=False)
        output = gr.Textbox(label="output", interactive=False, lines=20)
        event=button.click(main, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict, t, m, s, state], outputs=[output, limit, n], concurrency_limit=None)
        stop_btn.click(cancel, state)

interface.queue(concurrency_count=None)
interface.launch(share=True, show_error=True, show_api=False)

In [ ]:
#@title **Launch App (For Google Drive users)**
#@markdown Run this cell to launch the gradio GUI.<br>Note: if the tables have not been created yet, it takes about 3 to 4 minutes to launch the GUI.

import os
os.chdir('/content')

!wget https://downloads.python.org/pypy/pypy3.10-v7.3.16-linux64.tar.bz2
!tar -xf pypy3.10-v7.3.16-linux64.tar.bz2
!rm pypy3.10-v7.3.16-linux64.tar.bz2
os.environ['PATH']+=':/content/pypy3.10-v7.3.16-linux64/bin/'
!pip install gradio

from google.colab import drive
import subprocess
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive')
if not os.path.exists('/content/drive/MyDrive/RubiksSolver'):
    subprocess.run(['git', 'clone', 'https://github.com/or18/RubiksSolver.git'])
%cd /content/drive/MyDrive/RubiksSolver

subprocess.run(['pypy3.10', 'create_all_table.py'])

import RubiksSolver.solver as sv
import gradio as gr

def run_solver(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict):
    if rot is None:
        rot = "N"
    if restrict is None:
        restrict = "UDLRFB"
    if length is None:
        length=0
    command = ['pypy3.10', 'solver_for_app.py', operation, str(scramble), str(rot), str(slot1), str(slot2), str(slot3), str(slot4), str(length), restrict]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print("Command failed with return code:", result.returncode)
    return result.stdout

with gr.Blocks() as interface:
    gr.Markdown("# RubiksSolver")
    gr.Markdown("GitHub Repository: [GitHub](https://github.com/or18/RubiksSolver)")
    gr.Markdown("Google colab note book: [Google colab](https://colab.research.google.com/github/or18/RubiksSolver/blob/main/RubiksSolverApp.ipynb)")
    with gr.Tab('Cross & F2L'):
        gr.Markdown("### Solver for Cross, xcross, xxcross, xxxcross, xxxxcross, F2L, multi slotting, etc.")
        operation = gr.Textbox(value="F2L_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B' D L2 U F2 L2 U' B2 F2 D2 R2 B2 F R2 U2 R U2 B D L2 R'")
        rot = gr.Textbox(label="pre rotation", value="z2")
        slot1 = gr.Dropdown(choices=["False", "True"], label="solve BL slot or not", value="True")
        slot2 = gr.Dropdown(choices=["False", "True"], label="solve BR slot or not", value="False")
        slot3 = gr.Dropdown(choices=["False", "True"], label="solve FR slot or not", value="False")
        slot4 = gr.Dropdown(choices=["False", "True"], label="solve FL slot or not", value="False")
        length = gr.Number(label="max length", value=9)
        restrict = gr.Textbox(label="move restriction", value="UDLRFB")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer substeps'):
        gr.Markdown("### Solver for OLL, COLL, ZBLS, VHLS, etc.")
        operation = gr.Textbox(value="LL_substep_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="R2 B2 R2 U2 R' U2 R' B2 R2 U R' F' U F R")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Dropdown(choices=["False", "True"], label="solve LL CP or not", value="False")
        slot2 = gr.Dropdown(choices=["False", "True"], label="solve LL CO or not", value="True")
        slot3 = gr.Dropdown(choices=["False", "True"], label="solve LL EP or not", value="False")
        slot4 = gr.Dropdown(choices=["False", "True"], label="solve LL EO or not", value="True")
        length = gr.Number(label="max length", value=12)
        restrict = gr.Textbox(label="move restriction", value="UDRF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="U L2 D' B2 U B2 L2 B2 D F' D' B2 D F' U' F2")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=14)
        restrict = gr.Textbox(label="move restriction", value="URF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer & AUF'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_AUF_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B2 D2 L2 B2 R2 U' B2 D' L2 U L2 U' B F' D2 B F'")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=15)
        restrict = gr.Textbox(label="move restriction", value="URF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
interface.launch(share=True)

In [ ]:
#@title **Launch App (For People who don't use Google Drive)**
#@markdown Run this cell to launch the gradio GUI.<br>Note: if the tables have not been created yet, it takes about 3 to 4 minutes to launch the GUI.

import os
os.chdir('/content')

!wget https://downloads.python.org/pypy/pypy3.10-v7.3.16-linux64.tar.bz2
!tar -xf pypy3.10-v7.3.16-linux64.tar.bz2
!rm pypy3.10-v7.3.16-linux64.tar.bz2
os.environ['PATH']+=':/content/pypy3.10-v7.3.16-linux64/bin/'
!pip install gradio

import subprocess
if not os.path.exists('/content/RubiksSolver'):
    subprocess.run(['git', 'clone', 'https://github.com/or18/RubiksSolver.git'])
%cd /content/RubiksSolver

subprocess.run(['pypy3.10', 'create_all_table.py'])

import RubiksSolver.solver as sv
import gradio as gr

def run_solver(operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict):
    if rot is None:
        rot = "N"
    if restrict is None:
        restrict = "UDLRFB"
    if length is None:
        length=0
    command = ['pypy3.10', 'solver_for_app.py', operation, str(scramble), str(rot), str(slot1), str(slot2), str(slot3), str(slot4), str(length), restrict]
    result = subprocess.run(command, capture_output=True, text=True)
    if result.returncode != 0:
        print("Command failed with return code:", result.returncode)
    return result.stdout

with gr.Blocks() as interface:
    gr.Markdown("# RubiksSolver")
    gr.Markdown("GitHub Repository: [GitHub](https://github.com/or18/RubiksSolver)")
    gr.Markdown("Google colab note book: [Google colab](https://colab.research.google.com/github/or18/RubiksSolver/blob/main/RubiksSolverApp.ipynb)")
    with gr.Tab('Cross & F2L'):
        gr.Markdown("### Solver for Cross, xcross, xxcross, xxxcross, xxxxcross, F2L, multi slotting, etc.")
        operation = gr.Textbox(value="F2L_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B' D L2 U F2 L2 U' B2 F2 D2 R2 B2 F R2 U2 R U2 B D L2 R'")
        rot = gr.Textbox(label="pre rotation", value="z2")
        slot1 = gr.Dropdown(choices=["False", "True"], label="solve BL slot or not", value="True")
        slot2 = gr.Dropdown(choices=["False", "True"], label="solve BR slot or not", value="False")
        slot3 = gr.Dropdown(choices=["False", "True"], label="solve FR slot or not", value="False")
        slot4 = gr.Dropdown(choices=["False", "True"], label="solve FL slot or not", value="False")
        length = gr.Number(label="max length", value=9)
        restrict = gr.Textbox(label="move restriction", value="UDLRFB")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer substeps'):
        gr.Markdown("### Solver for OLL, COLL, ZBLS, VHLS, etc.")
        operation = gr.Textbox(value="LL_substep_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="R2 B2 R2 U2 R' U2 R' B2 R2 U R' F' U F R")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Dropdown(choices=["False", "True"], label="solve LL CP or not", value="False")
        slot2 = gr.Dropdown(choices=["False", "True"], label="solve LL CO or not", value="True")
        slot3 = gr.Dropdown(choices=["False", "True"], label="solve LL EP or not", value="False")
        slot4 = gr.Dropdown(choices=["False", "True"], label="solve LL EO or not", value="True")
        length = gr.Number(label="max length", value=12)
        restrict = gr.Textbox(label="move restriction", value="UDRF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="U L2 D' B2 U B2 L2 B2 D F' D' B2 D F' U' F2")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=14)
        restrict = gr.Textbox(label="move restriction", value="URF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
    with gr.Tab('Last Layer & AUF'):
        gr.Markdown("### Solver for PLL, ZBLL, 1LLL, last slots+last layer, etc.")
        operation = gr.Textbox(value="LL_AUF_solver", visible=False)
        scramble = gr.Textbox(label="scramble", value="B2 D2 L2 B2 R2 U' B2 D' L2 U L2 U' B F' D2 B F'")
        rot = gr.Textbox(label="pre rotation", value="")
        slot1 = gr.Textbox(value="False", visible=False)
        slot2 = gr.Textbox(value="False", visible=False)
        slot3 = gr.Textbox(value="False", visible=False)
        slot4 = gr.Textbox(value="False", visible=False)
        length = gr.Number(label="max length", value=15)
        restrict = gr.Textbox(label="move restriction", value="URF")
        button = gr.Button("start search")
        output = gr.Textbox(label="output")
        button.click(run_solver, inputs=[operation, scramble, rot, slot1, slot2, slot3, slot4, length, restrict], outputs=output)
interface.launch(share=True)